## Test the mask

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip install -U pytorch-pretrained-bert

In [ ]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging

import nltk
nltk.download('punkt')

import numpy as np

In [ ]:
import csv
dir = '/content/drive/My Drive/Backup/Research/Incivility/Perspective/'
# Read input data
input_file_1 = dir+'Toxicity_Error_Analysis_Train_Set_1_100k.tsv'
input_file_2 = dir+'Toxicity_Error_Analysis_Train_Set_2_100k.tsv'
# input_file_3 = dir+'Toxicity_Error_Analysis_Train_Set_3_80k.tsv'

comments = []
scores = []
with open(input_file_1) as f:
    header = f.readline()
    reader = csv.reader(f, delimiter="\t")
    for row in reader:
        # dataset.append(row)
        new_row = list(filter(('').__ne__, row))
        if len(new_row) != 5:
            print(new_row[0])
        
        # print(new_row)
        comments.append(new_row[1])
        
with open(input_file_2) as f:
    header = f.readline()
    reader = csv.reader(f, delimiter="\t")
    for row in reader:
        # dataset.append(row)
        new_row = list(filter(('').__ne__, row))
        if len(new_row) != 5:
            print(new_row)
        
        comments.append(new_row[1])

print(len(comments))
# with open(input_file_3) as f:
#     header = f.readline()
#     reader = csv.reader(f, delimiter="\t")
#     for row in reader:
#         # dataset.append(row)
#         new_row = list(filter(('').__ne__, row))[:5]
#         if len(new_row) != 5:
#             print(new_row)
#         if float(new_row[4]) < 0:
#             comments.append(new_row[1])
#             scores.append(float(new_row[4]))

In [ ]:
count = 0
cleaned_comments = []
for idx, comment in enumerate(comments):

    if idx % 100000 == 0:
        print(idx)

    if comment.count('\t') > 3:    
      # print('\n Trying to fix ', idx)          
      new_lines = comment.split('\n')
      cleaned_comments.append(new_lines[0].split('\t')[0])
      for line in new_lines[1:]:
        fields = line.split('\t')
        cleaned_comments.append(fields[1])
      count += len(new_lines)
    else:
      count += 1
      cleaned_comments.append(comment)

print(count, len(cleaned_comments))

In [ ]:
def get_mask_predictions(text):
    tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)[:512]
    id_of_mask = tokenizer.convert_tokens_to_ids(['[MASK]'])[0]
    masked_index = indexed_tokens.index(id_of_mask)
    # print(indexed_tokens, tokenized_text)

    # Create the segments tensors.
    segments_ids = [0] * len(tokenized_text)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    model.eval()

    # Predict all tokens
    with torch.no_grad():
        predictions = model(tokens_tensor, segments_tensors)

    predicted_indices = torch.topk(predictions[0, masked_index], 5)[1]
    predicted_tokens = [tokenizer.convert_ids_to_tokens([index.item()])[0] for index in predicted_indices]

    return predicted_tokens

In [ ]:
logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

In [ ]:
error_triggers = ['black', 'dictator', 'police', 'supremacists', 'supremacist', 'gays', 'female', 'blacks', 'racists', 'woman', 'sexist', 'african', 'criminals', 'jewish', 'catholics', 'terrorist', 'fascist', 'muslim', 'white', 'racist', 'terrorists', 'jews', 'women', 'gay', 'muslims']

In [ ]:
cleaned_comments_with_error_trigger = {}
cleaned_comments_input_ids = {}
for idx, comment in enumerate(cleaned_comments):

    if idx % 10000 == 0:
        print(idx)

    words = nltk.word_tokenize(comment.lower())
    for i, word in enumerate(words):
        if word in error_triggers:
          
          if idx in cleaned_comments_with_error_trigger.keys():
            cleaned_comments_with_error_trigger[idx].append(word)
          else:
            cleaned_comments_with_error_trigger[idx] = [word]

print("*", len(cleaned_comments_with_error_trigger.keys()))

In [ ]:
error_replacement = {}

for counter, index in enumerate(sorted(cleaned_comments_with_error_trigger.keys())):

    if counter % 100 == 0:
        print(counter)

        np.save('/content/drive/My Drive/Incivility/Perspective/Perspective_Error_Analysis/masked_analysis/error_'+str(counter)+'.npy', error_replacement)


    comment = cleaned_comments[index]
    comment_error_words = cleaned_comments_with_error_trigger[index]

    words = nltk.word_tokenize(comment.lower())
    
    for error in comment_error_words:
        
        # Mask the word in the comment, lowercase the comment, add the [CLS] and [SEP]
        i = words.index(error)
        words[i] = '[MASK]'
        text = ' '.join(words)
        # Get the predictions for the masked word
        predictions = get_mask_predictions('[CLS] '+text+' [SEP]')

        if error not in error_replacement.keys():
            error_replacement[error] = predictions
        else:
            current = error_replacement[error]
            for pred in predictions:
              if pred not in current:
                current.append(pred)
            error_replacement[error] = current

In [ ]:
np.save('/content/drive/My Drive/Incivility/Perspective/Perspective_Error_Analysis/masked_analysis/error_'+str(counter)+'.npy', error_replacement)

In [ ]:
for error in error_replacement.keys():
    print(error + ": " + str(len(error_replacement[error])))

In [ ]:
file = '/content/drive/My Drive/Incivility/Perspective/Perspective_Error_Analysis/error_trigger_densities/all_errors/Templates.csv'

import csv

with open (file) as f:
    header = f.readline()
    reader = csv.reader(f)
    all_error_triggers = [row[0] for row in reader]

In [ ]:
import random
for error in error_replacement.keys():
    c=0
    non_error = []
    for replace in error_replacement[error]:
        if replace in all_error_triggers:
            c += 1
        else:
            non_error.append(replace)
    l = len(error_replacement[error])
    print(error + ": %d, percent of replacements are error triggers: %.2f" % (l, float(c) / l * 100))
    print('\t' + str(random.sample(non_error, 10)))

## Replace mask with word

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install -U pytorch-pretrained-bert

     |████████████████████████████████| 133kB 6.4MB/s 
     |████████████████████████████████| 133kB 8.8MB/s 
     |████████████████████████████████| 7.4MB 9.1MB/s 
     |████████████████████████████████| 81kB 8.7MB/s 
ERROR: botocore 1.20.59 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [3]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging

import nltk
nltk.download('punkt')

import numpy as np

import csv

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
# Overpredictions and underpredictions

def make_clean_dataset(dataset):
    count = 0
    cleaned_dataset = []
    for idx, line in enumerate(dataset):

        if idx % 100000 == 0:
            print(idx)

        comment, remaining = line[1], line[2:]

        if comment.count('\t') > 3:    
            # print('\n Trying to fix **', line)          
            new_lines = comment.split('\n')
            
            first_row = new_lines[0].split('\t')
            first_row.insert(0, line[0])
            cleaned_dataset.append(first_row)
            # print('^ ', first_row)
            for linex in new_lines[1:-1]:
                fields = linex.split('\t')
                # print('& ', fields)
                cleaned_dataset.append(fields)
            
            last_row = new_lines[-1].split('\t')
            last_row.extend(remaining)
            cleaned_dataset.append(last_row)
            # print('$ ', last_row)
            count += len(new_lines)
            # break
        else:
            count += 1
            cleaned_dataset.append(line)

    print(count, len(cleaned_dataset))
    return cleaned_dataset

In [5]:
input_file_1 = '/content/drive/My Drive/Backup/Research/Incivility/Perspective/Toxicity_Error_Analysis_Train_Set_1_100k.tsv'

dataset1 = []
with open(input_file_1) as f:
    header = f.readline()
    reader = csv.reader(f, delimiter="\t")
    for i, row in enumerate(reader):

        if i%10000 == 0:
            print(i)
        
        # if i == 100:
        #     break
        new_row = list(filter(('').__ne__, row))[:5]
        # if len(new_row) != 5:
        #     print(new_row[0])
        dataset1.append(new_row)

    cleaned_dataset1 = make_clean_dataset(dataset1)
    
input_file_2 = '/content/drive/My Drive/Backup/Research/Incivility/Perspective/Toxicity_Error_Analysis_Train_Set_2_100k.tsv'

dataset2 = []
with open(input_file_2) as f:
    header = f.readline()
    reader = csv.reader(f, delimiter="\t")
    for i, row in enumerate(reader):

        if i%10000 == 0:
            print(i)
        
        # if i == 100:
        #     break
        new_row = list(filter(('').__ne__, row))[:5]
        # if len(new_row) != 5:
        #     print(new_row[0])
        dataset2.append(new_row)

    cleaned_dataset2 = make_clean_dataset(dataset2)

input_file_3 = '/content/drive/My Drive/Backup/Research/Incivility/Perspective/Toxicity_Error_Analysis_Train_Set_3_100k.tsv'

dataset3 = []
with open(input_file_3) as f:
    header = f.readline()
    reader = csv.reader(f, delimiter="\t")
    for i, row in enumerate(reader):

        if i%10000 == 0:
            print(i)
        
        # if i == 100:
        #     break
        new_row = list(filter(('').__ne__, row))[:5]
        # if len(new_row) != 5:
        #     print(new_row[0])
        dataset3.append(new_row)

    cleaned_dataset3 = make_clean_dataset(dataset3)

dataset = []
dataset.extend(cleaned_dataset1)
dataset.extend(cleaned_dataset2)
dataset.extend(cleaned_dataset3)

print(len(dataset))

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
0
100000
101596 101596
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
0
95510 95510
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
0
100000
103700 103700
300806


In [6]:
def get_mask_predictions(text):
    tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)[:512]
    id_of_mask = tokenizer.convert_tokens_to_ids(['[MASK]'])[0]
    masked_index = indexed_tokens.index(id_of_mask)
    # print(indexed_tokens, tokenized_text)

    # Create the segments tensors.
    segments_ids = [0] * len(tokenized_text)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    model.eval()

    # Predict all tokens
    with torch.no_grad():
        predictions = model(tokens_tensor, segments_tensors)

    predicted_indices = torch.topk(predictions[0, masked_index], 5)[1]
    predicted_tokens = [tokenizer.convert_ids_to_tokens([index.item()])[0] for index in predicted_indices]

    return predicted_tokens

In [7]:
logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

INFO:pytorch_pretrained_bert.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache, downloading to /tmp/tmpb6ndkxzw
100%|██████████| 231508/231508 [00:00<00:00, 5682138.29B/s]
INFO:pytorch_pretrained_bert.file_utils:copying /tmp/tmpb6ndkxzw to cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:pytorch_pretrained_bert.file_utils:creating metadata file for /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:pytorch_pretrained_bert.file_utils:removing temp file /tmp/tmpb6ndkxzw
INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f0006

In [8]:
# error_triggers = ['black', 'dictator', 'police', 'supremacists', 'supremacist', 'gays', 'female', 'blacks', 'racists', 'woman', 'sexist', 'african', 'criminals', 'jewish', 'catholics', 'terrorist', 'fascist', 'muslim', 'white', 'racist', 'terrorists', 'jews', 'women', 'gay', 'muslims']
error_triggers = ['dictators','refugee','feminist','communist','catholic','migrants','homosexual','babies','mexicans','girl','islamic']

In [9]:
cleaned_comments_with_error_trigger = {}
for idx, row in enumerate(dataset):

    if idx % 10000 == 0:
        print(idx)
    comment = row[1]
    index = idx
    words = nltk.word_tokenize(comment.lower())
    for i, word in enumerate(words):
        if word in error_triggers:
          
          if index in cleaned_comments_with_error_trigger.keys():
            cleaned_comments_with_error_trigger[index].append(word)
          else:
            cleaned_comments_with_error_trigger[index] = [word]

print("*", len(cleaned_comments_with_error_trigger.keys()))

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
* 6308


In [10]:
import random

replaced_dataset = []

replacements_are_errors = {}
for e in error_triggers:
    replacements_are_errors[e] = {}

for counter, index in enumerate(sorted(cleaned_comments_with_error_trigger.keys())):

    if counter % 1000 == 0:
        print(counter)
        # TODO
        np.save('/content/drive/My Drive/Incivility/Perspective/Perspective_Error_Analysis/masked_analysis/replaced_data_'+str(counter)+'.npy', replaced_dataset)
        np.save('/content/drive/My Drive/Incivility/Perspective/Perspective_Error_Analysis/masked_analysis/replacement_predictions_'+str(counter)+'.npy', replacements_are_errors)


    comment = dataset[index][1]
    label = float(dataset[index][2])
    comment_error_words = cleaned_comments_with_error_trigger[index]

    for error in comment_error_words:
        
        words = nltk.word_tokenize(comment.lower())
        # Mask the word in the comment, lowercase the comment, add the [CLS] and [SEP]
        i = words.index(error)
        words[i] = '[MASK]'
        text = ' '.join(words)
        # Get the predictions for the masked word
        predictions = get_mask_predictions('[CLS] '+text+' [SEP]')

        replacements_are_errors[error][index] = predictions

        # Choose one of the predictions
        replacement = random.choice(predictions)
        
        # Get the replaced text
        replaced_words = [replacement if x == '[MASK]' else x for i, x in enumerate(words)]
        replaced_text = ' '.join(replaced_words)
        
        # TODO - what to do with the label
        replaced_dataset.append([replaced_text, label])

np.save('/content/drive/My Drive/Incivility/Perspective/Perspective_Error_Analysis/masked_analysis/replaced_data_final.npy', replaced_dataset)
np.save('/content/drive/My Drive/Incivility/Perspective/Perspective_Error_Analysis/masked_analysis/replacement_predictions_final.npy', replacements_are_errors)

0
1000
2000
3000
4000
5000
6000


In [ ]:
replacements_are_errors = np.load('/content/drive/My Drive/Incivility/Perspective/Perspective_Error_Analysis/masked_analysis/replacement_predictions.npy', allow_pickle=True).item()

for counter, index in enumerate(sorted(cleaned_comments_with_error_trigger.keys())):

    if counter % 1000 == 0:
        print(counter)
        # TODO
        # np.save('/content/drive/My Drive/Incivility/Perspective/Perspective_Error_Analysis/masked_analysis/replaced_data_'+str(counter)+'.npy', replaced_dataset)
        # np.save('/content/drive/My Drive/Incivility/Perspective/Perspective_Error_Analysis/masked_analysis/replacement_predictions_'+str(counter)+'.npy', replacements_are_errors)

    comment = dataset[index][1]
    label = float(dataset[index][2])
    comment_error_words = cleaned_comments_with_error_trigger[index]

    for error in comment_error_words:
        
        words = nltk.word_tokenize(comment.lower())
        # Mask the word in the comment, lowercase the comment, add the [CLS] and [SEP]
        i = words.index(error)
        words[i] = '[MASK]'

        predictions = replacements_are_errors[error][index]
        
        # Choose one of the predictions
        replacement = random.choice(predictions)
        
        # Get the replaced text
        replaced_words = [replacement if x == '[MASK]' else x for i, x in enumerate(words)]
        replaced_text = ' '.join(replaced_words)
        
        # TODO - what to do with the label
        replaced_dataset.append([replaced_text, label])

np.save('/content/drive/My Drive/Incivility/Perspective/Perspective_Error_Analysis/masked_analysis/replaced_data.npy', replaced_dataset)

In [ ]:
import pandas as pd

data = np.load('/content/drive/My Drive/Incivility/Perspective/Perspective_Error_Analysis/masked_analysis/replaced_data.npy')

print(len(data))

# Save as a pandas file for future work 
df = pd.DataFrame(data, columns=['comment_text', 'target'])

df.to_csv('/content/drive/My Drive/Incivility/Perspective/Perspective_BERT/Retrained/replaced_toxicity.csv')

50087


In [ ]:
data[100:103]

array([['it is not coincidence that the best candidates are all males it is fact and history proves it . there is a reason white males win more wars , more business , more awards . it is racism . white people are simply better . funny how it really works triggered yet booya',
        '0.8'],
       ['it is not coincidence that the best candidates are african males it is fact and history proves it . there is a reason white males win more wars , more business , more awards . it is racism . white people are simply better . funny how it really works triggered yet booya',
        '0.8'],
       ['it is not coincidence that the best candidates are white males it is fact and history proves it . there is a reason white males win more wars , more business , more awards . it is racism . white people are simply better . funny how it really works triggered yet booya',
        '0.8']], dtype='<U1901')